In [210]:
import random
import numpy as np

In [211]:
DICT = {(0,0) : 'U', (0,1) : 'A', (0,2) : 'T', (0,3) : 'A',
        (1,0) : 'C', (1,1) : 'G', (1,2) : 'F', (1,3) : 'G',
        (2,0) : 'D', (2,1) : 'S', (2,2) : 'R', (2,3) : 'S',
        (3,0) : 'C', (3,1) : 'G', (3,2) : 'F', (3,3) : 'G'}
INV_DICT = {v: k for k, v in DICT.items()}

WORLD_WIDTH = 30
WORLD_LENGTH = 30
SMOOTHNESS = 1
# Smoothness is on a scale from 0 to 1, where 0 is not smooth and 1 is smooth

In [38]:
for t in range(3):
    for p in range(3):
        print(DICT[(t,p)],end=" ")
    print()

U A T 
C G F 
D S R 


In [43]:
biomes = np.empty(shape=(WORLD_WIDTH,WORLD_HEIGHT), dtype=str)
temp = 1
precip = 1
for i in range(WORLD_WIDTH):
    for j in range(WORLD_HEIGHT):
        update_temp = 0
        update_precip = 0
        if random.randint(0,1):
            update_temp = round(random.uniform(-1,1))
        else:
            update_precip = round(random.uniform(-1,1))
        temp = (temp + update_temp) % 4
        precip = (precip + update_precip) % 4
        biomes[i,j] = DICT[(temp, precip)]

In [206]:
def get_adjacent_biomes(i, j, biomes):
    adjacent_biomes = np.empty(shape=4, dtype=tuple)
    if i > 0:
        adjacent_biomes[0] = INV_DICT[biomes[i-1,j]] if biomes[i-1,j] != "" else None
    if i < WORLD_WIDTH - 1:
        adjacent_biomes[1] = INV_DICT[biomes[i+1,j]] if biomes[i+1,j] != "" else None
    if j > 0:
        adjacent_biomes[2] = INV_DICT[biomes[i,j-1]] if biomes[i,j-1] != "" else None
    if j < WORLD_HEIGHT - 1:
        adjacent_biomes[3] = INV_DICT[biomes[i,j+1]] if biomes[i,j+1] != "" else None
    return adjacent_biomes
        
def print_map(biomes):
    for x in range(WORLD_WIDTH):
        for y in range(WORLD_HEIGHT):
            print(biomes[x, y], end="")
        print()
        
def gen_unrestricted_biome():
    temp = random.randint(0,3)
    precip = random.randint(0,3)
    return temp, precip

def gen_restricted_biome(adjacent_biomes, weak=False):
    biome_adjacent = np.random.choice(adjacent_biomes[adjacent_biomes != None])
    if not weak and -1 * SMOOTHNESS < random.triangular(-1,1) < SMOOTHNESS:
        return biome_adjacent
    temp, precip = gen_unrestricted_biome()
    while not is_biome_legal(temp, precip, adjacent_biomes, weak):
        temp, precip = gen_unrestricted_biome()
    return temp, precip
    
def is_biome_legal(temp, precip, adjacent_biomes, weak=False):
    for biome in adjacent_biomes:
        if not biome == None:
            adjacent_temp, adjacent_precip = biome
            if not weak:
                if abs(adjacent_temp - temp) > 1:
                    return False
                else:
                    if abs(adjacent_temp - temp) + abs(adjacent_precip - precip) == 2:
                        return False
            else:
                if abs(adjacent_temp - temp) == 2 and abs(adjacent_precip - precip == 2):
                    return False
    return True

def is_biome_possible(adjacent_biomes):
    if np.all(adjacent_biomes == None):
        return True
    for temp in range(3):
        for precip in range(3):
            if is_biome_legal(temp, precip, adjacent_biomes):
                return True
    return False

In [208]:
biomes = np.empty(shape=(WORLD_WIDTH,WORLD_HEIGHT), dtype=str)
weak_counter = 0
while np.any(biomes == ""):
    i = random.randint(0, WORLD_WIDTH - 1)
    j = random.randint(0, WORLD_HEIGHT - 1)
    
    if biomes[i,j] != "":
        continue
    adjacent_biomes = get_adjacent_biomes(i, j, biomes)
    biome_adjacent = next((biome for biome in adjacent_biomes if biome is not None), None)
    weak_counter += int(not is_biome_possible(adjacent_biomes))

    temp, precip = gen_unrestricted_biome() if biome_adjacent == None else gen_restricted_biome(adjacent_biomes, weak=(not is_biome_possible(adjacent_biomes)))
    biomes[i,j] = DICT[(temp, precip)]

print(f"Map Generated Successfully. {weak_counter}/{WORLD_WIDTH * WORLD_HEIGHT} weak biomes.")
print_map(biomes)

Map Generated Successfully. 189/900 weak biomes.
DAAAAAFGGGGGGFTTGGSUUGDDCCGGFF
DDAAAACUGGGGGSTAGGUCGGGDDDGGFF
DDDATGGAGGGSGGTUFAAGGFFDDGGGGU
RRAAAGGAFDGSSSGUUFAUTFFGGGGRTA
RRUUARRAFAACSCFRAGAUSFGGGTGSTA
RRUUGSRRGURCCCGAAFSGGSGGSTRCSS
TTTTFSRRGDFFFFGAARRRRRRAFFCCCR
TUUGGSRRGDFSFFFGSGCRGDRAFDUTGC
AGCCCCFGGGFSFFFFCCCCRDDDDDDSFC
ADGSSSSCGGGGFFFCCCCRRCDDGGGFFT
TTTGSDDCGGGGSGCCCFFFSCCCGGGGRA
GSRGSDDSSGCAAGCCCCFSSSCCSGGGFA
CCCCSGGSGGFATGGCCCFDSUDDRRRCAA
CCCCCGGFGGGAGGGDDRADUUDDRRRRAA
CCSSAAAFFFDGGGGGDRSDTUDDRRRAAA
CSSSTUUGRRRRDGGGRTSCCAAARRAAAA
SGSTTUUUUUDDDGSTTTTFGSAARRAAAS
SGGSSGGTFTRDDDDDRRTAGGFDRRCSSF
SUSSSDGUCCRTTTDDFATAGCFCGGFFFF
AASSDDGFCCGATRDDFAAATCCCGGGSFF
AARRRDGGGGGDCRSSAGCGGGFGGGCSSF
AARRRSADDSGGGTGGUCGGGAAGCCCCDD
DGRRGGGSSSSGSSGGGGDGGUUGGCCCDD
FFFDGGSSSGGGGDGGGRDDGUTGGCCDDD
CFFGGGGCGGGDDDDGUTDDGGFGCCSSSD
FCASGCGRGGDDDDDFUTDDRRSSCCFFGC
FUAUCCFRGGDDRRGAAUUDDASSSGAATC
FFFGGGGGGGGGRRGGRCCDGCSSTRRSSS
FFGGGGSCGGGGGGGGGUAGGAAATTFFSS
RGGGGSSSSGGCAFGGAAAGGCAAARCFSS
